In [9]:
using Damysos,HDF5,CairoMakie,CSV,DataFrames,ProgressLogging,TerminalLoggers,LoggingExtras
global_logger(TerminalLogger())

SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(23) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

In [6]:
function load_hdf5_files(path::String)
	allfiles = String[]
	for (root, dirs, files) in walkdir(path)
		for file in files
			file[end-4:end] == ".hdf5" && file[1:4] == "conv" && push!(allfiles,
					joinpath(root, file))
		end
	end
	@info "$(length(allfiles)) files found."
	return allfiles
end

load_hdf5_files (generic function with 1 method)

In [7]:
dtpath = "/home/atuin/b228da/b228da10/zeta-M-sweep/dt"
dtfiles = load_hdf5_files(dtpath)

┌ Info: 1024 files found.
└ @ Main /home/hpc/b228da/b228da10/Damysos.jl/jupyter/runtime-analysis.ipynb:9


1024-element Vector{String}:
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 125 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 140 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 141 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 140 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 140 bytes ⋯ "ta=0.1_dt_PowerLawTest_dt.hdf5"
 ⋮
 "/home/atuin/b228da/b228da10/zet" ⋯ 168 bytes ⋯ "753032_dt_PowerLawTest_dt.hdf5"
 "/home/atuin/b228da/b228da10/zet" ⋯ 167 bytes ⋯ "753032_dt_PowerL

In [18]:
function collectresults(files::Vector{String})

	goodfiles = String[]
	coord = Tuple{Float64,Float64}[]
	zetas = Float64[]
	ms = Float64[]
	dts = Float64[]
	dkxs = Float64[]
	rtols = Float64[]
	atols = Float64[]
	runtimes = Float64[]

	@progress for f in files

		m = match(r"M=([0-9.]+)_zeta=([0-9.]+)", basename(f))
		if isnothing(m)
			@warn "No regex match for $(pattern) in $f"
			continue
		end
		
		M = parse(Float64, m.captures[1])
		z = parse(Float64, m.captures[2])

		if (M,z) ∈ coord
			@warn "Duplicate at at zeta = $z M = $M.Skipping..."
			continue
		end

		h5open(f,"r") do file
			if "testresult" ∈ keys(file)
				if successful_retcode(ReturnCode.T(read(file["testresult"],"retcode")))
					params = NumericalParams2d(read(file["testresult"],"last_params"))

					push!(rtols,read(file["testresult"],"achieved_rtol"))
					push!(atols,read(file["testresult"],"achieved_atol"))
					push!(dkxs,params.dkx)
					push!(dts,params.dt)
					push!(goodfiles,f)
					push!(ms,M)
					push!(zetas,z)
					push!(coord,(M,z))
					push!(runtimes,read(file["testresult"],"elapsed_time_sec"))
				else
					@info "Result not converged at at zeta = $z M = $M in $f"
				end

			else
				@warn "testresult not found at zeta = $z M = $M in $f"
			end
		end

	end

	return DataFrame(
		:path => goodfiles,
		:zeta => zetas,
		:M => ms,
		:dt => dts,
		:dkx => dkxs,
		:rtol => rtols,
		:atol => atols,
		:runtime => runtimes)
end

collectresults (generic function with 1 method)

In [19]:
dtres = collectresults(dtfiles)
maximum(dtres.runtime)

Progress:   0%|                                         |  ETA: N/A
Progress:   1%|▎                                        |  ETA: 0:00:02
Progress:   1%|▌                                        |  ETA: 0:00:02
Progress:   2%|▊                                        |  ETA: 0:00:02
Progress:   2%|█                                        |  ETA: 0:00:02
Progress:   3%|█▎                                       |  ETA: 0:00:02
Progress:   4%|█▌                                       |  ETA: 0:00:02
Progress:   4%|█▋                                       |  ETA: 0:00:02
Progress:   5%|█▉                                       |  ETA: 0:00:02
Progress:   5%|██▏                                      |  ETA: 0:00:02
Progress:   6%|██▍                                      |  ETA: 0:00:02
Progress:   6%|██▋                                      |  ETA: 0:00:01
Progress:   7%|██▉                                      |  ETA: 0:00:01
Progress:   8%|███▏                                     |  ETA: 0:00

173.37846622799998

In [20]:
mean(x::Vector{<:Number}) = sum(x) / length(x)

mean (generic function with 1 method)

In [21]:
@info ""

77.73308751852052